<a href="https://colab.research.google.com/github/lluc2397/ScrapDataroma/blob/main/ScrapDataroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate'
}

In [ ]:
site = 'https://www.dataroma.com'


In [ ]:
def get_investors_accronym(site):
    url = requests.get(f'{site}/m/managers.php', headers=headers).content
    soup = bs(url, 'html.parser')

    main_table = soup.find('table', id='grid')

    all_td = main_table.find_all('td', class_='man')
    l_as = []
    for t in all_td[1:]:
      investor_name = t.text
      l_as.append(t.find('a', href=True)['href'].split('=')[1])
    return l_as

In [ ]:
def get_activity(accro):
  investor_url = f'{site}/m/m_activity.php?m={accro}&typ=a&L=2'
  url = requests.get(investor_url, headers=headers).content
  soup = bs(url, 'html.parser')

  pages = [div.text for div in soup.find('div', id="pages").find_all('a')][1:-1]

  for page in pages[:1]:
    investor_url = f'{site}/m/m_activity.php?m={accro}&typ=a&L={page}'
    url = requests.get(investor_url, headers=headers).content
    soup = bs(url, 'html.parser')

    for td in soup.find_all('td')[5:]:
      info = td.text
      attrs = td.attrs
      clase = attrs.get('class')

      if td.find('b') is not None:
        quarter = td.text.split(' ')[0][1:]
        year = td.text.split(' ')[1][-4:]
        continue # Quarter and year
      
      if clase:
        if clase[0] == 'stock':

          # print(info.split('-')[0].strip()) # Ticker
          
          continue

        elif clase[0] == 'buy' or clase[0] == 'sell':
          movement = None
          is_new = False
          if 'Add' in info or 'Buy' in info:
            if 'Buy' in info:
              is_new = True
              print(is_new)
            movement = 1
          elif 'Reduce' in info or 'Sell' in info:
            movement = 2
          if movement is not None:
            mv_activity = info.split(' ')[1][:-1]
            print('mv_activity', mv_activity)
            
            continue # activity
          else:
            share_change = info.replace(',', '')
            print('share_change', share_change)
            continue # share change
      else:
        print('last', info)
        pass # Change portfolio

In [ ]:
def get_history():
  url = f'{site}/m/hist/hist.php?f=hcmax&s=PFE'
  url = requests.get(url, headers=headers).content
  soup = bs(url, 'html.parser')
  table = pd.read_html(url)[0]
  table['Activity'] = table['Activity'].fillna('Hold')
  table = table.fillna(0)
  for index, content in table.iterrows():
    info = content['Period']
    ticker = info.split('-')[0].strip() # Ticker
    name = info.split('-')[1].strip()
    print(ticker)
    print(name)
    print('*'*100)

get_history()

2021
4
****************************************************************************************************
2021
3
****************************************************************************************************
2021
2
****************************************************************************************************
2021
1
****************************************************************************************************
2020
4
****************************************************************************************************
2020
3
****************************************************************************************************
2020
2
****************************************************************************************************
2020
1
****************************************************************************************************
2019
4
****************************************************************************************************
2019
3
*********************